## package install

```
$ pip install oandapyV20
$ pip install backtesting==0.1.7
$ pip install TA-Lib
$ brew install ta-lib
$ conda install pandas
$ conda install configparser
$ conda install requests
```

In [1]:
import configparser
import datetime
import pandas as pd
from oandapyV20 import API

config = configparser.ConfigParser()
config.read("../oanda_config.txt")
api_key = config["Practice"]["api_key"]
oanda_api = API(api_key, "practice")

In [2]:
import oandapyV20.endpoints.instruments as instruments
def get_candles(instrument, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument, params)
    resp = oanda_api.request(r)
    data = []
    for candle in resp["candles"]:
        data.append([
            datetime.datetime.fromisoformat(candle["time"][:19]),
            candle["mid"]["o"],
            candle["mid"]["h"],
            candle["mid"]["l"],
            candle["mid"]["c"],
            candle["volume"],
        ])
    return data

In [3]:
UTC = datetime.timezone.utc
date_from = datetime.datetime(2020, 11, 3, 0, 0, tzinfo=UTC)
date_to = datetime.datetime(2020, 11, 9, 23, 55, tzinfo=UTC)
candles = get_candles("USD_JPY", date_from, date_to, "M5")

# DataFrame 型へ変換
df = pd.DataFrame(candles)
df.columns = ["Time", "Open", "High", "Low", "Close", "Volume"]
# カラムの型を float へ変換
df["Open"] = df["Open"].astype(float)
df["High"] = df["High"].astype(float)
df["Low"] = df["Low"].astype(float)
df["Close"] = df["Close"].astype(float)

print(df)
df.dtypes

1604361600
1604966100
hogeaaaaaaaaa
destance604500
hoge1604361600
                    Time     Open     High      Low    Close  Volume
0    2020-11-03 14:00:00  104.656  104.670  104.645  104.654     472
1    2020-11-03 14:05:00  104.655  104.683  104.644  104.675     482
2    2020-11-03 14:10:00  104.676  104.680  104.662  104.668     459
3    2020-11-03 14:15:00  104.670  104.678  104.607  104.613     641
4    2020-11-03 14:20:00  104.614  104.633  104.600  104.604     540
...                  ...      ...      ...      ...      ...     ...
1432 2020-11-10 13:30:00  105.368  105.386  105.352  105.358     484
1433 2020-11-10 13:35:00  105.360  105.402  105.358  105.398     611
1434 2020-11-10 13:40:00  105.396  105.396  105.324  105.336     622
1435 2020-11-10 13:45:00  105.337  105.378  105.332  105.375     463
1436 2020-11-10 13:50:00  105.374  105.374  105.306  105.308     499

[1437 rows x 6 columns]


Time      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaCrossStrategy(Strategy):

    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10

    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)

    def next(self):
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()

/Users/inoue/.conda/envs/market_analysis/lib/python3.8/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
from backtesting import Backtest

df_ = df.copy()
bt_ema = Backtest(df_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
bt_ema.run()
bt_ema.plot()


<ipython-input-5-31d2e8b303c2>:4: UserWarning: Data index is not datetime. Assuming simple periods.
  bt_ema = Backtest(df_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
